Preprocessing 

Configure the paths and the libraries used for preprocessing data

In [7]:
import pandas as pd
import os

print(os.getcwd())


c:\Users\Adria Espinoza\FIB\mds\Q2\ML\ml-project\src


Extract data

In [14]:

df = pd.read_csv("./data/raw_files/train.csv")

print("Dataframe columns")
print(df.columns)

print("Dataframe rows")
print(len(df))

Dataframe columns
Index(['language', 'sentence', 'n', 'edgelist', 'root'], dtype='object')
Dataframe rows
10500
